In [1]:
# from torch.autograd import Variable
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import torch
import pickle
import torchvision
from torchvision import transforms
import torchvision.datasets as dset
from torchvision import transforms
from torch.utils.data import DataLoader
from torch.autograd import Variable
import matplotlib.pyplot as plt
import time
import numpy as np
import sys
from collections import deque
import os

In [2]:
individuals_file = "individuals"
vocabulary       = []
with open(individuals_file,'r') as f:
    for line in f:
        vocabulary.append(line.strip().split()[1])
vocabulary_size = len(vocabulary)

In [3]:
infer_file       = "relation_infer"
relations_file   = "relations"
        
wirelation     =  1
worelation     =  0
subject_index  =  1
relation_index =  2
object_index   =  3

with open(relations_file,'r') as f:
    relation_number = 0
    for line in f:
        relation_number += 1

whole_dict     = {new_list: [] for new_list in range(relation_number)} 

with open(infer_file,'r') as f:
    for line in f:
        relation = (line.strip())
        # label, subject, object
        if relation.split()[0]== "+":
            whole_dict[int(relation.split()[relation_index])].append([wirelation, int(relation.split()[subject_index]), int(relation.split()[object_index])])
        else:
            whole_dict[int(relation.split()[relation_index])].append([worelation, int(relation.split()[subject_index]), int(relation.split()[object_index])])
            



In [4]:
def get_input_layer(word_idx):
    x = torch.zeros(vocabulary_size).float()
    x[word_idx] = 1.0
    return x

In [5]:
class Onto_MLP(nn.Module):
    def __init__(self, embedding, voca_size, hidden_size):
        self.voca_size   = voca_size
        self.embedding   = embedding
        self.hidden_size = hidden_size
        super(Onto_MLP, self).__init__()
        self.conv = nn.Sequential(
            nn.Linear(self.voca_size, self.embedding),
            nn.Tanh(),
        )
        self.liner = nn.Sequential(nn.Linear(self.embedding*2, self.hidden_size), nn.Sigmoid())
        self.out   = nn.Sequential(nn.Linear(self.hidden_size, 1), nn.Sigmoid())
        
    def forward_one(self, x):
        x = self.conv(x)
        return x

    def forward(self, x1, x2):
        out1 = self.forward_one(x1)
        out2 = self.forward_one(x2)
        dis = torch.cat((out1, out2), dim=0)
        diss = self.liner(dis)
        out = self.out(diss)
        return out

In [6]:
def binary_acc(y_pred, y_test):
    y_pred_tag = torch.round((y_pred))
    correct_results_sum = (y_pred_tag == y_test).sum().float()
    acc = correct_results_sum/y_test.shape[0]
    acc = torch.round(acc * 100)
    return acc

In [7]:
embedding_dims = 10
hidden_size    = 15
learning_rate  = 0.0001

In [8]:
loss_fn   = torch.nn.BCEWithLogitsLoss(size_average=True)
net       = Onto_MLP(embedding_dims, vocabulary_size, hidden_size)
optimizer = torch.optim.Adam(net.parameters(),lr = learning_rate )
optimizer.zero_grad()

/workdir2/home/sam/.pyenv/versions/3.7ver/lib/python3.7/site-packages/torch/nn/_reduction.py:43: UserWarning: size_average and reduce args will be deprecated, please use reduction='mean' instead.
  warnings.warn(warning.format(ret))


In [9]:
correct     = 0
print_epoch = 50
train_loss  = []
loss_val    = 0
epoch       = 500
threshold   = 0.5

net.train()

time_start  = time.time()

for iteration in range(1, epoch):
    
    for label, su, ob in whole_dict[5]:
        
        optimizer.zero_grad()
        su_onehot = Variable(get_input_layer(su)).float()
        ob_onehot = Variable(get_input_layer(ob)).float()
        output    = net.forward(su_onehot, ob_onehot)
        loss      = loss_fn(output, torch.tensor([float(label)]))
        loss_val  += loss.item()
        
        loss.backward()
        optimizer.step()
        
        output    = (output > threshold).float()
        correct   += (output == label).float().sum()
        
    if iteration % print_epoch ==0 and iteration != 0:
        
        ae        = (loss_val/len(whole_dict[5])/print_epoch)
        loss_val  = 0
        accuracy  = 100 * (correct / print_epoch) / len(whole_dict[5])
        correct   = 0
        
        print("Accuracy = {}, Loss = {}".format(accuracy, ae))
        
print ("Finish training.")        


Accuracy = 91.01388549804688, Loss = 0.7608086673046153
Accuracy = 95.83333587646484, Loss = 0.6939151055448585
Accuracy = 95.83333587646484, Loss = 0.6931759958051973
Accuracy = 95.83333587646484, Loss = 0.6931483149114582
Accuracy = 95.83333587646484, Loss = 0.6931472301648722
Accuracy = 95.83333587646484, Loss = 0.6931471824645996
Accuracy = 95.83333587646484, Loss = 0.6931471824645996
Accuracy = 95.83333587646484, Loss = 0.6931471824645996
Accuracy = 95.83333587646484, Loss = 0.6931471824645996
Finish training.
